### Example script for importing WRF CTRL 3D files using date and location information 
### of California flood events (identified using Stage IV data)
### author: Erin Dougherty
### contact: edough@rams.colostate.edu
### date created: June 10, 2020

In [1]:
### import packages

import math
import numpy as np
import pandas as pd
import matplotlib as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import netCDF4 as nc
from netCDF4 import Dataset, num2date
from datetime import datetime, date, timedelta
import glob
import xarray as xr
from wrf import getvar, ALL_TIMES

#### import list of CA AR floods

In [2]:
pd.set_option('display.max_columns', None)

ar_floods = pd.read_csv('/glade/work/doughert/flooddata/REU_code_files/CA_AR_floods_CTRL_good.csv', delimiter=',',header=0, 
                         dtype={'enddate': str,'startdate': str}, usecols=range(0,18))

ar_floods = ar_floods.drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
#print(ar_floods.iloc[0:3])


#### get list of year-month for start and end dates of episodes to open only those WRF files

In [3]:
import unicodedata

### convert flood start and end date string to date object
start_date_fflood = pd.to_datetime(ar_floods['startdate'], format = '%Y%m%d.%H')
start_date_fflood = start_date_fflood.reset_index(drop=True)

end_date_fflood = pd.to_datetime(ar_floods['enddate'], format = '%Y%m%d.%H')
end_date_fflood = end_date_fflood.reset_index(drop=True)

#### convert lat and lon_ep from list of strings to list of arrays

In [5]:
lat_ep_fflood_int = []
lon_ep_fflood_int = []

for i in range(len(ar_floods)):
    list_lat = ar_floods['lat_10deg'][i][1:-1].split(",")
    list_lon = ar_floods['lon_10deg'][i][1:-1].split(",")
    lat_ep_fflood_int.append([float(i) for i in list_lat])
    lon_ep_fflood_int.append([float(i) for i in list_lon])
    
lat10 = []
lon10 = []

for i in range(len(ar_floods)):  
    if ar_floods['flood_type'].iloc[i]=='slow':
        lat10.append(lat_ep_fflood_int[i][10:-10])
        lon10.append(lon_ep_fflood_int[i][10:-10])
    else:
        lat10.append(lat_ep_fflood_int[i])
        lon10.append(lon_ep_fflood_int[i])

## get list of start and ending lats to subset wrf files 
lat_ep_fflood_st = [item[0] for item in lat10] 
lat_ep_fflood_end = [item[-1] for item in lat10]

lon_ep_fflood_st = [item[0] for item in lon10] 
lon_ep_fflood_end = [item[-1] for item in lon10]


#### get start and end time of floods to match (and subset) times in WRF file

In [7]:
import itertools

date_fflood_st = start_date_fflood.dt.strftime('%Y-%m-%d_%H:%M:%S')
date_fflood_end = end_date_fflood.dt.strftime('%Y-%m-%d_%H:%M:%S')

date_fflood_st_dt =[]
date_fflood_end_dt =[]

### change into datetime object
for i in range(len(date_fflood_st)):
    date_fflood_st_dt.append(datetime.strptime(date_fflood_st[i],'%Y-%m-%d_%H:%M:%S'))
    
for i in range(len(date_fflood_end)):
    date_fflood_end_dt.append(datetime.strptime(date_fflood_end[i],'%Y-%m-%d_%H:%M:%S'))

#### get list of all dates and hours between start and end time for all episodes

In [8]:
def perdelta(start, end, delta):
    curr = start
    while curr <= end:
        yield curr
        curr += delta
        
date_list_fflood = [] 
episode_list = []
lon_ep_st_list = []
lon_ep_end_list = []
lat_ep_st_list = []
lat_ep_end_list = []

### get list of all hours between start and end time for each episode       
for i in range(len(date_fflood_end_dt)):
    for result in perdelta(date_fflood_st_dt[i], date_fflood_end_dt[i], timedelta(hours=1)):
        date_list_fflood.append(result.strftime('%Y-%m-%d_%H:%M:%S'))
        episode_list.append(ar_floods['Episodeid'].iloc[i])
        lon_ep_st_list.append(lon_ep_fflood_st[i])
        lon_ep_end_list.append(lon_ep_fflood_end[i])
        lat_ep_st_list.append(lat_ep_fflood_st[i])
        lat_ep_end_list.append(lat_ep_fflood_end[i])



date_3h = ('00:00:00', '03:00:00', '06:00:00', '09:00:00', '12:00:00', '15:00:00', '18:00:00', '21:00:00')

date_list_3h_fflood= []
episode_list_3h_fflood= []
lon_ep_st_3h= []
lon_ep_end_3h= []
lat_ep_st_3h= []
lat_ep_end_3h= []

### remove hours that are not multiples of threes (0,3,6,9,12,15,18)
for i in range(len(date_list_fflood)):
    if date_list_fflood[i].endswith((date_3h)):
        date_list_3h_fflood.append(datetime.strptime(date_list_fflood[i],'%Y-%m-%d_%H:%M:%S'))
        episode_list_3h_fflood.append(episode_list[i])
        lon_ep_st_3h.append(lon_ep_st_list[i])
        lon_ep_end_3h.append(lon_ep_end_list[i])
        lat_ep_st_3h.append(lat_ep_st_list[i])
        lat_ep_end_3h.append(lat_ep_end_list[i])


### only get YYYYMMDD for each 3h increment in flood
date_list_3h_ymd = []
for i in range(len(date_list_3h_fflood)):
    date_list_3h_ymd.append(datetime.strftime(date_list_3h_fflood[i],'%Y%m%d'))


#### import in chunks, to prevent kernel from crashing

In [9]:
#episode_list_3h_fflood_sub = episode_list_3h_fflood[0:157]
#episode_list_3h_fflood_sub = episode_list_3h_fflood[157:330]
#episode_list_3h_fflood_sub = episode_list_3h_fflood[330:500]
episode_list_3h_fflood_sub = episode_list_3h_fflood[500:608]

#date_list_3h_fflood_sub = date_list_3h_fflood[0:157]
#date_list_3h_fflood_sub = date_list_3h_fflood[157:330]
#date_list_3h_fflood_sub = date_list_3h_fflood[330:500]
date_list_3h_fflood_sub = date_list_3h_fflood[500:608]

#date_list_3h_ymd_sub = date_list_3h_ymd[0:157]
#date_list_3h_ymd_sub = date_list_3h_ymd[157:330]
#date_list_3h_ymd_sub = date_list_3h_ymd[330:500]
date_list_3h_ymd_sub = date_list_3h_ymd[500:608]

#### specify variable to look for

In [10]:
import os.path

#var = 'QVAPOR'
#var = 'U'
#var = 'V'
var = 'P'


#### define bounds outside of which to nan values (using West Coast + part of Pacific Ocean)

In [11]:
ar_lo_lat = 30
ar_hi_lat = 45
ar_w_lon = -134
ar_e_lon = -114

### import WRF data

In [12]:
var_list = []

### Find files that match start and end year/month of flood and subset based on flood duration
for c, item in enumerate(date_list_3h_ymd_sub):   
    print(c)
    for name in glob.glob(os.path.join('/glade/collections/rda/data/ds612.0/CTRL3D/**/wrf3d_d01_CTRL_'+var+ '_*')): 
        if item in name:
            wrf_match_fflood = xr.open_dataset(name)
            # pull only times of floods
            eu= wrf_match_fflood[var].sel(Time = slice(date_list_3h_fflood_sub[c], date_list_3h_fflood_sub[c]))
            # nan out all values outside of AR domain (i.e., W Coast + part of Pacific)
            pcpnan = eu.where((ar_w_lon <= eu.XLONG) & 
                              (eu.XLONG <= ar_e_lon)
                            & (ar_lo_lat <= eu.XLAT) &  
                              (eu.XLAT <= ar_hi_lat), 
                               drop=False)
            var_list.append(pcpnan)

0
1
2
3
4
5
6


KeyboardInterrupt: 

#### get index where episode ids are the same

In [13]:
## get list of unique episodes
indexes = np.unique(episode_list_3h_fflood_sub, return_index=True)[1]

unique_episode_1h_fflood = [episode_list_3h_fflood_sub[index] for index in sorted(indexes)]

ep_index = []
for x in unique_episode_1h_fflood:
    test = np.where(episode_list_3h_fflood_sub==x)
    #test = np.where(episode_list_fflood_fall==x)
    ep_index.append(test)


### get index where unique episode starts and ends
tst2 = [item[0] for item in ep_index]
ep_st_index= [item[0] for item in tst2 ]
ep_end_index= [item[-1] for item in tst2 ]

startdate_ep = [date_list_3h_fflood_sub[index] for index in sorted(ep_st_index)]
enddate_ep = [date_list_3h_fflood_sub[index] for index in sorted(ep_end_index)]


#### make sure variables aren't taking up too much memory

In [14]:

import sys
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') \
        and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('ar_floods', 60623),
 ('date_list_fflood', 14680),
 ('episode_list', 14680),
 ('lat_ep_end_list', 14680),
 ('lat_ep_st_list', 14680),
 ('lon_ep_end_list', 14680),
 ('lon_ep_st_list', 14680),
 ('date_list_3h_fflood', 5496),
 ('date_list_3h_ymd', 5496),
 ('episode_list_3h_fflood', 5496),
 ('lat_ep_end_3h', 5496),
 ('lat_ep_st_3h', 5496),
 ('lon_ep_end_3h', 5496),
 ('lon_ep_st_3h', 5496),
 ('date_fflood_end', 3572),
 ('date_fflood_st', 3572),
 ('date_list_3h_fflood_sub', 928),
 ('date_list_3h_ymd_sub', 928),
 ('episode_list_3h_fflood_sub', 928),
 ('list_lat', 576),
 ('list_lon', 576),
 ('end_date_fflood', 512),
 ('start_date_fflood', 512),
 ('date_fflood_end_dt', 432),
 ('date_fflood_st_dt', 432),
 ('lat10', 432),
 ('lat_ep_fflood_end', 432),
 ('lat_ep_fflood_int', 432),
 ('lat_ep_fflood_st', 432),
 ('lon10', 432),
 ('lon_ep_fflood_end', 432),
 ('lon_ep_fflood_int', 432),
 ('lon_ep_fflood_st', 432),
 ('Dataset', 400),
 ('date', 400),
 ('timedelta', 400),
 ('indexes', 160),
 ('getvar', 1

In [ ]:
del ar_floods
del date_list_3h_fflood_sub
del date_list_3h_ymd_sub
del episode_list_3h_fflood_sub
del date_list_3h_fflood
del episode_list_3h_fflood
del lat_ep_end_3h
del lat_ep_st_3h
del lon_ep_st_3h
del lon_ep_end_3h
del list_lat
del list_lon
del end_date_fflood,
del start_date_fflood
del date_fflood_end_dt
del date_fflood_st_dt
del date_list_fflood
del episode_list
del lat_ep_end_list
del lat_ep_st_list
del lon_ep_end_list
del lon_ep_st_list
del date_list_3h_ymd
del date_fflood_end
del date_fflood_st
#del eu_list
del lat10
del lat_ep_fflood_end
del lat_ep_fflood_int
del lat_ep_fflood_st
del lon10
del lon_ep_fflood_end
del lon_ep_fflood_int
del lon_ep_fflood_st

#### combine arrays that are part of same episode

In [ ]:
wrf_list_ep = []

for i in range(len(tst2)):
    print(i)
    combine = xr.concat([elts for elts in eu_list[ep_st_index[i]:ep_end_index[i]+1]], dim='Time')
    wrf_list_ep.append(combine)

#### calcualte mean (across time) for each variable and episode in IVT eqn. 

In [ ]:
ep_mean_var = []

for i in range(len(tst2)):
    meanvar= (wrf_list_ep[i].mean(axis=0, skipna=False)).to_dataset(name='var%s' %i)
    ep_mean_var.append(meanvar)

### create a dataset to export

In [ ]:
ep_mean_var_mg = xr.merge(ep_mean_var)

#### export dataset

In [ ]:
ep_mean_var_mg.to_netcdf(path='/glade/scratch/doughert/CA_AR_output/CA_AR_mean_P_CTRL4.nc', mode='w', format='NETCDF4', compute=False)